In [1]:
import hmac
import hashlib
import base64

import xml.etree.ElementTree as ET
from comarmor.xml.utils import beautify_xml

In [2]:
def obscure_permissions(path):
    tree = ET.parse(path)
    root = tree.getroot()
    permissions = root.find('permissions')
    for grant in permissions.findall('grant'):
        obscure_grant(grant, digestmod=hashlib.sha256)
    return tree


def obscure_grant(grant, digestmod):
    subject_name = grant.find('subject_name')
    for rule in grant.findall('*/domains/..'):
        obscure_rule(
            rule=rule,
            key=subject_name.text.encode('utf-8'),
            digestmod=digestmod)


def obscure_rule(rule, key, digestmod):
    obscure_topics(rule, key, digestmod)
    obscure_partitions(rule, key, digestmod)
    obscure_data_tags(rule, key, digestmod)


def obscure_topics(rule, key, digestmod):
    for element in rule.iter('topic'):
        if not element.text.isspace():
            element.text = encoded_digest(
                key=key,
                msg=element.text,
                digestmod=digestmod)


def obscure_partitions(rule, key, digestmod):
    for element in rule.iter('partition'):
        if not element.text.isspace():
            element.text = encoded_digest(
                key=key,
                msg=element.text,
                digestmod=digestmod)


def obscure_data_tags(rule, key, digestmod):
    for tag in ['name', 'value']:
        for element in rule.iter(tag=tag):
            if not element.text.isspace():
                element.text = encoded_digest(
                    key=key,
                    msg=element.text,
                    digestmod=digestmod)

def encoded_digest(key, msg, digestmod):
    digest_bytes = hmac.HMAC(
        key=key,
        msg=msg.encode('utf-8'),
        digestmod=digestmod).digest()
    digest_base64 = base64.b64encode(digest_bytes)
    return digest_base64.decode('utf-8')

In [3]:
permissions_file_path = '/home/ruffsl/Desktop/keymint_ws/build/talker/permissions.xml'

# import urllib.request
# some_url = 'https://www.omg.org/spec/DDS-SECURITY/20170901/omg_shared_ca_permissions_example.xml'
# with urllib.request.urlopen(some_url) as url:
#     permissions_file_path = url.read()

In [4]:
tree = obscure_permissions(permissions_file_path)

In [5]:
print(beautify_xml(tree.getroot()))

<?xml version="1.0" encoding="utf-8"?>
<dds>
  <permissions>
    <grant name="talker">
      <subject_name>CN=talker</subject_name>
      <validity>
        <not_before>2013-10-26T00:00:00</not_before>
        <not_after>2023-10-26T22:45:30</not_after>
      </validity>
      <allow_rule>
        <domains>
          <id_range>
            <min>0</min>
            <max>42</max>
          </id_range>
        </domains>
        <subscribe>
          <topics>
            <topic>kdt1TIh7MLHoSSFtsV8Sen/y40Ry4Wr6OVc8QubGAeM=</topic>
            <topic>vD4DoW8GEI2lkw19IaK9TQJH0JblZGqpzSn7mYpjg1E=</topic>
            <topic>JrJzmUDG+omEeGMS4fv3w2I9hG1zGekmTq89+FJATY8=</topic>
            <topic>vF86GtW36eea9mkHcrrOEpnKOIXmYcPLBsR1vhjC2/k=</topic>
            <topic>YL7AQRBinh5jN5KwaYDbN4uAxGYq2rI5lglGlg1R3IU=</topic>
            <topic>ZRl3CJBrF3CeCD2mtJm4BnlEsk4od4rQwcubXNXYAfo=</topic>
            <topic>cDNPKcMY1Zr+ncWxBL8BFaIdeeroHcfRsyVbpcYlFeI=</topic>
            <topic>TrGYu3Qm301rp/L3e

In [6]:
root = tree.getroot()
digest_str = root.find('permissions/grant/allow_rule/subscribe/topics/topic').text
digest_bytes = base64.decodebytes(digest_str.encode('utf-8'))

In [7]:
subject_name = root.find('permissions/grant/subject_name')
topic = 'rt/clock'
digestmod = hashlib.sha256
test_digest_bytes = hmac.HMAC(
    key=subject_name.text.encode('utf-8'),
    msg=topic.encode('utf-8'),
    digestmod=digestmod).digest()

hmac.compare_digest(digest_bytes, test_digest_bytes)

True